In [2]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

In [3]:
cudnn.benchmark = True
cudnn.deterministic = False

In [4]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [9]:
opt = get_config("config_files/en_filtered_config.yaml")

from dataset import hierarchical_dataset, AlignCollate, Batch_Balanced_Dataset
train_dataset = Batch_Balanced_Dataset(opt)

--------------------------------------------------------------------------------
dataset_root: sample_data
opt.select_data: ['test']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    sample_data	 dataset: test
sample_data/test
sub-directory:	/test	 num samples: 100
num total samples of test: 100 x 1.0 (total_data_usage_ratio) = 100
num samples of test per batch: 32 x 1.0 (batch_ratio) = 32
--------------------------------------------------------------------------------
Total_batch_size: 32 = 32
--------------------------------------------------------------------------------


In [ ]:
train(opt, amp=False)